# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


S. Kraus  ->  S. Kraus  |  ['S. Kraus']
Arxiv has 76 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.16899


extracting tarball to tmp_2507.16899...

 done.


G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']


Found 167 bibliographic references in tmp_2507.16899/aa54986-25.bbl.
Issues with the citations
syntax error in line 606: '=' expected
Retrieving document from  https://arxiv.org/e-print/2507.17315
extracting tarball to tmp_2507.17315... done.
Retrieving document from  https://arxiv.org/e-print/2507.17641


extracting tarball to tmp_2507.17641...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.17670
extracting tarball to tmp_2507.17670...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.17705
extracting tarball to tmp_2507.17705... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.16899-b31b1b.svg)](https://arxiv.org/abs/2507.16899) | **X-SHYNE: X-Shooter spectra of young exoplanet analogs II. Presentation and analysis of the full library**  |
|| S. Petrus, et al. -- incl., <mark>G. Chauvin</mark>, <mark>M. Ravet</mark>, <mark>P. Mollière</mark> |
|*Appeared on*| *2025-07-24*|
|*Comments*| *28 pages, 18 figures, 6 tables*|
|**Abstract**|            The X-SHYNE library is a homogeneous sample of 43 medium-resolution (R=8000) infrared (0.3-2.5um) spectra of young (<500Myr), low-mass (<20Mjup), and cold (Teff=600-2000K) isolated brown dwarfs and wide-separation companions observed with the VLT/X-Shooter instrument. To characterize our targets, we performed a global comparative analysis. We first applied a semi-empirical approach. By refining their age and bolometric luminosity, we derived key atmospheric and physical properties, such as Teff, mass, surface gravity (g), and radius, using the evolutionary model COND03. These results were then compared with the results from a synthetic analysis based on three self-consistent atmospheric models. To compare our spectra with these grids we used the Bayesian inference code ForMoSA. We found similar Lbol estimates between both approaches, but an underestimated Teff from the cloudy models, likely due to a lack of absorbers that could dominate the J and H bands of early L. We also observed a discrepancy in the log(g) estimates, which are dispersed between 3.5 and 5.5 dex for mid-L objects. We interpreted this as a bias caused by a range of rotational velocities leading to cloud migration toward equatorial latitudes, combined with a variety of viewing angles that result in different observed atmospheric properties (cloud column densities, atmospheric pressures, etc.). Finally, while providing robust estimates of [M/H] and C/O for individual objects remains challenging, the X-SHYNE library globally suggests solar values, which are consistent with a formation via stellar formation mechanisms. This study highlights the strength of homogeneous datasets in performing comparative analyses, reducing the impact of systematics, and ensuring robust conclusions while avoiding over-interpretation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.17705-b31b1b.svg)](https://arxiv.org/abs/2507.17705) | **The Small Separation A-Star Companion Population: Tentative Signatures of Enhanced Multiplicity with Primary Mass**  |
|| M. D. Furio, et al. -- incl., <mark>S. Kraus</mark> |
|*Appeared on*| *2025-07-24*|
|*Comments*| *14 pages, 3 figures, 6 tables, Accepted by ApJ July 22 2025*|
|**Abstract**|            We present updated results from our near-infrared long-baseline interferometry (LBI) survey to constrain the multiplicity properties of intermediate-mass A-type stars within 80 pc. Previous adaptive optics surveys of A-type stars are incomplete at separations $<$ 20au. Therefore, a LBI survey allows us to explore separations previously unexplored. Our sample consists of 54 A-type primaries with estimated masses between 1.44-2.93 M$_{\odot}$ and ages 10-790 Myr, which we observed with the MIRC-X and MYSTIC instruments at the CHARA Array. We use the open source software CANDID to detect two new companions, seven in total, and we performed a Bayesian demographic analysis to characterize the companion population. We find the separation distribution consistent with being flat, and we estimate a power-law fit to the mass ratio distribution with index -0.13$^{+0.92}_{-0.95}$ and a companion frequency of 0.25$^{+0.17}_{-0.11}$ over mass ratios 0.1-1.0 and projected separations 0.01-27.54au. We find a posterior probability of 0.53 and 0.04 that our results are consistent with extrapolations based on previous models of the solar-type and B-type companion population, respectively. Our results suggest that the close companion population to A-type stars is comparable to that of solar-types and that close companions to B-type stars are potentially more frequent which may be indicative of increased disk fragmentation for stars $\gtrsim$ 3M$_{\odot}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.17315-b31b1b.svg)](https://arxiv.org/abs/2507.17315) | **Prospects for searching for sterile neutrinos in dynamical dark energy cosmologies using joint observations of gravitational waves and $γ$-ray bursts**  |
|| L. Feng, T. Han, J.-F. Zhang, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-24*|
|*Comments*| *13 pages, 4 figures*|
|**Abstract**|            In the era of third-generation (3G) gravitational-wave (GW) detectors, GW standard siren observations from binary neutron star mergers provide a powerful tool for probing the expansion history of the universe. Since sterile neutrinos can influence cosmic evolution by modifying the radiation content and suppressing structure formation, GW standard sirens offer promising prospects for constraining sterile neutrino properties within a cosmological framework. Building on this, we investigate the prospects for detecting sterile neutrinos in dynamical dark energy (DE) models using joint observations from 3G GW detectors and a future short gamma-ray burst detector, such as a THESEUS-like telescope. We consider three DE models: the $w$CDM, holographic DE (HDE), and Chevallier-Polarski-Linder (CPL) models. Our results show that the properties of DE can influence the constraints on sterile neutrino parameters. Moreover, the inclusion of GW data significantly improves constraints on both sterile neutrino parameters and other cosmological parameters across all three models, compared to the current limits derived from CMB+BAO+SN (CBS) observations. When GW data are included into the CBS dataset, a preference for $\Delta N_{\rm eff} > 0$ emerges at approximately the $1\sigma$ level in the $w$CDM and CPL models, and reaches about $3\sigma$ in the HDE model. Moreover, the upper limits on $m_{\nu,{\rm sterile}}^{\rm eff}$ are reduced by approximately 13%, 75%, and 3% in the $w$CDM, HDE, and CPL models, respectively.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.17641-b31b1b.svg)](https://arxiv.org/abs/2507.17641) | **A deep search for radio pulsations from the 1.3 M$_{\odot}$ compact-object binary companion of young pulsar PSR J1906+0746**  |
|| <mark>Y. Wang</mark>, J. v. Leeuwen |
|*Appeared on*| *2025-07-24*|
|*Comments*| *8 pages, 11 figures. Accepted by A&A*|
|**Abstract**|            Double pulsar systems offer unrivaled advantages for the study of both astrophysics and fundamental physics. But only one has been visible: PSR J0737$-$3039; and its component pulsar B has now rotated out of sight due to the general-relativistic effect of geodetic precession. We know, though, that these precession cycles can also pivot pulsars into sight, and that this precession occurs at similar strength in PSR J1906+0746. That source is a young, unrecycled radio pulsar, orbiting a compact object with mass $\sim $1.32 M$_{\odot}$. This work presents a renewed campaign to detect radio pulsations from this companion, two decades after the previous search. Two key reasons driving this reattempt are the possibility that the companion radio beam has since precessed into our line of sight, and the improved sensitivity now offered by the FAST radio telescope. In 28 deep observations, we did not detect a credible companion pulsar signal. After comparing the possible scenarios, we conclude the companion is still most likely a pulsar that is not pointing at us. We next present estimates for the sky covered by such systems throughout their precession cycle. We find that for most system geometries, the all-time beaming fraction is unity, i.e., observers in any direction can see the system at some point. We conclude it is still likely that PSR J1906+0746 will be visible as a double pulsar in the future.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.17670-b31b1b.svg)](https://arxiv.org/abs/2507.17670) | **Mapping ground-based coronagraphic images to Helioprojective-Cartesian coordinate system by image registration**  |
|| F. Sha, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-24*|
|*Comments*| **|
|**Abstract**|            A few ground-based solar coronagraphs have been installed in western China for observing the low-layer corona in recent years. However, determining the Helioprojective Coordinates for the coronagraphic data with high precision is an important but challenging step for further research with other multi-wavelength data. In this paper, we propose an automatic coronal image registration method that combines local statistical correlation and feature point matching to achieve accurate registration between ground-based coronal green-line images and space-based 211 Å images. Then, the accurate field of view information of the coronal green-line images can be derived, allowing the images to be mapped to the Helioprojective Cartesian Coordinates with an accuracy of no less than 0.1''. This method has been extensively validated using 100 days of coronal data spanning an 11-year period, demonstrating its broad applicability to ground-based coronagraphs equipped with green-line observations. It significantly enhances the scientific value of ground-based coronal data, enabling comprehensive studies of coronal transient activities and facilitating the joint analysis of data from multiple instruments. Additionally, it holds potential for future applications in improving the pointing accuracy of coronagraphs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.16899/./Figures/xshyne_sequence_full.png', 'tmp_2507.16899/./Figures/lum.png', 'tmp_2507.16899/./Figures/tem.png']
copying  tmp_2507.16899/./Figures/xshyne_sequence_full.png to _build/html/
copying  tmp_2507.16899/./Figures/lum.png to _build/html/
copying  tmp_2507.16899/./Figures/tem.png to _build/html/
exported in  _build/html/2507.16899.md
    + _build/html/tmp_2507.16899/./Figures/xshyne_sequence_full.png
    + _build/html/tmp_2507.16899/./Figures/lum.png
    + _build/html/tmp_2507.16899/./Figures/tem.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mic}{\ensuremath{\upmu}m\xspace}$
$\newcommand{\Teff}{\ensuremath{\mathrm{T_{eff}}}\xspace}$
$\newcommand{\fsed}{\ensuremath{\mathrm{f_{sed}}}\xspace}$
$\newcommand{\kzz}{\ensuremath{\mathrm{K_{ZZ}}}\xspace}$
$\newcommand{\Rjup}{\ensuremath{\mathrm{R_{Jup}}}\xspace}$
$\newcommand{\Mjup}{\ensuremath{\mathrm{M_{Jup}}}\xspace}$
$\newcommand{\Lbol}{\ensuremath{\mathrm{L_{bol}}}\xspace}$
$\newcommand{\Rlam}{\ensuremath{\mathrm{R_{\uplambda}}}\xspace}$
$\newcommand{\chr}{\ensuremath{\mathrm{\chi^{2}_{r}}}\xspace}$
$\newcommand{\ab}[1]{\textcolor{red}{AB: #1}}$</div>



<div id="title">

# X-SHYNE: X-Shooter spectra of young exoplanet analogs$\thanks{Based on observations collected at the European Organization for Astronomical Research in the Southern Hemisphere under ESO programs 0101.C-0290; 0102.C-0121; 0103.C-0231; 0104.C-0094; 111.24PX.001; and 114.27B1.001}$: II. Presentation and analysis of the full library

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.16899-b31b1b.svg)](https://arxiv.org/abs/2507.16899)<mark>Appeared on: 2025-07-24</mark> -  _28 pages, 18 figures, 6 tables_

</div>
<div id="authors">

S. Petrus, et al. -- incl., <mark>G. Chauvin</mark>, <mark>M. Ravet</mark>, <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** The characterization of exoplanetary spectra is a crucial step in understanding the chemical and physical processes shaping their atmospheres and constraining their formation and evolutionary history. The X-SHYNE library is a homogeneous sample of 43 medium-resolution ( $\Rlam$  $\sim$ 8000) infrared (0.3–2.5 $\mic$ ) spectra of young ( $<$ 500 Myr), low-mass ( $<$ 20 $\Mjup$ ), and cold ( $\Teff$  $\sim$ 600–2000 K) isolated brown dwarfs and wide-separation companions observed with the VLT/X-Shooter instrument. To characterize our targets, we performed a global comparative analysis. We first applied a semi-empirical approach. By refining their age and bolometric luminosity, we derived key atmospheric and physical properties, such as $\Teff$ , mass, surface gravity (g), and radius, using the evolutionary model \texttt{COND03} . These results were then compared with the results from a synthetic analysis based on three self-consistent atmospheric models: the cloudy models \texttt{Exo-REM} and \texttt{Sonora Diamondback} , and the cloudless model \texttt{ATMO} . To compare our spectra with these grids we used the Bayesian inference code \texttt{ForMoSA} . We found similar $\Lbol$ estimates between both approaches, but an underestimated $\Teff$ from the cloudy models, likely due to a lack of absorbers that could dominate the J and H bands of early L. We also observed a discrepancy in the log(g) estimates, which are dispersed between 3.5 and 5.5 dex for mid-L objects. We interpreted this as a bias caused by a range of rotational velocities leading to cloud migration toward equatorial latitudes, combined with a variety of viewing angles that result in different observed atmospheric properties (cloud column densities, atmospheric pressures, etc.). This interpretation is supported by the correlation of the color anomaly $\Delta$ (J-K) of each object with log(g) and the parameter $\fsed$ that drives the sedimentation of the clouds. Finally, while providing robust estimates of [ M/H ] and C/O for individual objects remains challenging, the X-SHYNE library globally suggests solar values, which are consistent with a formation via stellar formation mechanisms. This study highlights the strength of homogeneous datasets in performing comparative analyses, reducing the impact of systematics, and ensuring robust conclusions while avoiding over-interpretation.

</div>

<div id="div_fig1">

<img src="tmp_2507.16899/./Figures/xshyne_sequence_full.png" alt="Fig4" width="100%"/>

**Figure 4. -** Sequence of the X-SHYNE spectra. The integrated flux between 1.2 and 1.3 $\mic$ normalized the spectra, and an offset was applied. The color of each spectrum indicates its spectral type. Several zooms are given in Figures \ref{fig:sequence_VIS_wav}, \ref{fig:sequence_Y_wav}, \ref{fig:sequence_J_wav}, \ref{fig:sequence_H_wav}, and \ref{fig:sequence_K_wav}. (*fig:sequence_full_wav*)

</div>
<div id="div_fig2">

<img src="tmp_2507.16899/./Figures/lum.png" alt="Fig8" width="100%"/>

**Figure 8. -** $\Lbol$ estimated from each model grid and for each spectral range using the Stefan-Boltzmann law, based on the estimated $\Teff$(see Figure \ref{fig:tem}) and radius (see Figure \ref{fig:rad}). Objects with a negative color anomaly are represented by small light dots, while those with a positive color anomaly are shown as large deep dots. The $\Lbol$ values derived from the reconstructed SED (see Section \ref{sec:semiempirical_Lbol}) are indicated by red crosses. (*fig:lum*)

</div>
<div id="div_fig3">

<img src="tmp_2507.16899/./Figures/tem.png" alt="Fig9" width="100%"/>

**Figure 9. -** Same as in Figure \ref{fig:lum}, but for the $\Teff$. The boundaries of each model grid are represented by the filled colored regions. (*fig:tem*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.16899"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

149  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
